In [1]:
# The RSA.py file implements the RSA code from challenge 41, which I'll reuse here

from RSA import RSAClient, RSAServer, BIT_STRENGTH
from matplotlib import pyplot as plt

In [2]:
# First just verify everything is working...

message = b'A'

server = RSAServer()
pubkey = server.GetPubkey()
client = RSAClient(**pubkey)

ciphertext = client.Encrypt(message)

assert server.Decrypt(ciphertext) == message

In [3]:
# Now make the oracle

class Oracle(RSAServer):
    def IsMessageEven(self, c):
        m = self.DecryptInt(c)
        assert m == m%self.n
        return False if m%2 else True
    
server = Oracle()
pubkey = server.GetPubkey()
client = RSAClient(**pubkey)

ciphertext = client.Encrypt(10)
server.IsMessageEven(ciphertext)
server.DecryptBytes(ciphertext)

b'\n'

In [4]:
# Now get the secret message

import base64

message = base64.decodebytes(b'VGhhdCdzIHdoeSBJIGZvdW5kIHlvdSBkb24ndCBwbGF5IGFyb3VuZCB3aXRoIHRoZSBGdW5reSBDb2xkIE1lZGluYQ==')
ciphertext = client.Encrypt(message)

In [5]:
# The modulus is prime, which means it must be odd
# If I'm multiplying by powers of two... the result must be even
# UNTIL it wraps the modulus, at which point it MUST be odd

from math import log as ln

# My initial attempt was only valid for even messages... here is a way around that
if not server.IsMessageEven(ciphertext):
    needs_doubling = True
    ciphertext *= client.Encrypt(2)
else:
    needs_doubling = False

n = server.n
assert n%2
assert server.IsMessageEven(ciphertext) # This current method will only work for even messages


# First we need to establish an upper bound so we don't wrap the modulus twice

multiplier = 1
while True:
    multiplier *= 2
    test_value = ciphertext * client.Encrypt(multiplier)
    
    if server.IsMessageEven(test_value):
        # Keep going
        continue
    else:
        break

# So now we have an upper bound

hi = multiplier
lo = 2

nbits = ln(hi) / ln(2)

for itr in range( 1+int(nbits) ):
    mid = ( hi + lo ) // 2
    test_value = ciphertext * client.Encrypt(mid)
    
    if server.IsMessageEven(test_value):
        lo = mid
    else:
        hi = mid
        
    # Print the guess at each attempt    
    guess = n//lo
    guess //= 2 if needs_doubling else 1
    print( guess.to_bytes(byteorder='big', length = 2*BIT_STRENGTH//8).replace(b'\x00', b'') )


b'\x9b\x92)\x97\xda\xe7\xa6\xe4B\x0e\x87q\xa6A\x89\xa8\r\xa6l?\xdcjta\x8e(\xb8Qy\xbcR\xcd\xe1\xb9(Ni\t\x18\xf8\xbaz\xc1c\xde\xc1?1Q\xbc\x02\xf0\x7f\xe5\xfb6c\x86p\x9b\xc5>?\xd5?\xd37'
b'g\xb6\xc6e<\x9aoB\xd6\xb4ZK\xc4+\xb1\x1a\xb3\xc4H*\x92\xf1\xa2\xeb\xb4\x1b%\x8b\xa6}\x8c\x89A&\x1a\xde\xf0\xb0\xbbP|Q\xd6B\x94\x80\xd4\xcb\x8b\xd2\xac\xa0UC\xfc\xce\xed\x04K\x12\x83~\xd58\xd57z'
b'X\xe5\xce\x9f\xea\xcd\x83\xf0%\xbf(\xd3:n\x97\xcd\xbe\xa8=\xdbYag\x13,\xa9\x8d\xe5j"xu\xa5\x8e`,\xceNWi\x8f!\x93\x14\x7fI\xda\xf7\x9ckod\xdb^\xd8\xb1]qd\xebL#\x920\xb6\xc1\xd6'
b'X\xe5\xce\x9f\xea\xcd\x83\xf0%\xbf(\xd3:n\x97\xcd\xbe\xa8=\xdbYag\x13,\xa9\x8d\xe5j"xu\xa5\x8e`,\xceNWi\x8f!\x93\x14\x7fI\xda\xf7\x9ckod\xdb^\xd8\xb1]qd\xebL#\x920\xb6\xc1\xd6'
b'U\xd5\x0e\x1e\xd1\r\x0c\xa1>\xed\x88\x85R\xe6]\x9a\x83\x1e\x06\xc2!U7a\xf6(\x1f\x12x!H*\xf8\x1f\x88\xff\x1fw\xc7( C\xba%r\x12W\xd4\x97\x06\xa0\x84\xb0~\xe2\xe06\xe9\x11\xfd\xaa\x9d\xee@\xb0t\x88'
b'U\xd5\x0e\x1e\xd1\r\x0c\xa1>\xed\x88\x85R\xe6]\x9a\x83\x1e\x0